In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf


In [ ]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised = True)

train_dataset, test_dataset = dataset['train'], dataset['test']

encoder = info.features['text'].encoder

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteBR7LOD/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteBR7LOD/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

padded_shapes = ([None], ())

train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes = padded_shapes)

test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes = padded_shapes)

In [ ]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size, 64),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(1, activation= 'sigmoid')])

model.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(1e-4),
              metrics = ['accuracy'])

history = model.fit(train_dataset, epochs = 5, validation_data = test_dataset, validation_steps = 30)

Epoch 1/5
391/391 [==============================] - 132s 311ms/step - loss: 0.6273 - accuracy: 0.6249 - val_loss: 0.4789 - val_accuracy: 0.7922
Epoch 2/5
391/391 [==============================] - 120s 304ms/step - loss: 0.3319 - accuracy: 0.8684 - val_loss: 0.3474 - val_accuracy: 0.8620
Epoch 3/5
391/391 [==============================] - 120s 306ms/step - loss: 0.2483 - accuracy: 0.9090 - val_loss: 0.3212 - val_accuracy: 0.8750
Epoch 4/5
391/391 [==============================] - 121s 306ms/step - loss: 0.2055 - accuracy: 0.9270 - val_loss: 0.3284 - val_accuracy: 0.8714
Epoch 5/5
391/391 [==============================] - 120s 303ms/step - loss: 0.1809 - accuracy: 0.9374 - val_loss: 0.3363 - val_accuracy: 0.8661


In [ ]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(sentence, pad):
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))
  return predictions

sample_text = ('This movie was awesome. The acting was incredible. Highly recommended')
predictions = sample_predict(sample_text, pad = True) * 100

print('probability of this being a positive review %.2f'% predictions)

sample_text2 = ('This movie was so so. The acting was mediocre. Kind of whack')
predictions_text2 = sample_predict(sample_text2, pad= True) * 100
print('probability of this being a positive review %.2f'%predictions_text2)


probability of this being a positive review 85.45
probability of this being a positive review 31.82


In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(encoder.vocab_size, 64),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer= tf.keras.optimizers.Adam(1e-4),
              metrics= ['accuracy'])

history = model.fit(train_dataset, epochs = 5, validation_data=test_dataset, validation_steps= 35)

Epoch 1/5
391/391 [==============================] - 239s 591ms/step - loss: 0.6434 - accuracy: 0.5993 - val_loss: 0.4601 - val_accuracy: 0.7915
Epoch 2/5
391/391 [==============================] - 229s 583ms/step - loss: 0.3601 - accuracy: 0.8607 - val_loss: 0.3573 - val_accuracy: 0.8598
Epoch 3/5
391/391 [==============================] - 229s 583ms/step - loss: 0.2633 - accuracy: 0.9070 - val_loss: 0.3241 - val_accuracy: 0.8732
Epoch 4/5
391/391 [==============================] - 230s 585ms/step - loss: 0.2176 - accuracy: 0.9281 - val_loss: 0.3464 - val_accuracy: 0.8679
Epoch 5/5
391/391 [==============================] - 228s 580ms/step - loss: 0.1884 - accuracy: 0.9402 - val_loss: 0.3645 - val_accuracy: 0.8692


def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(sentence, pad, model):
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))
  return predictions

sample_text = ('This movie was awesome. The acting was incredible. Highly recommended')
predictions = sample_predict(sample_text, pad = True, model) * 100

print('probability of this being a positive review %.2f'% predictions)

sample_text2 = ('This movie was so so. The acting was mediocre. Kind of whack')
predictions_text2 = sample_predict(sample_text2, pad= True, model) * 100
print('probability of this being a positive review %.2f'%predictions_text2)
